In [5]:
# Librerias
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import json
import re

In [6]:
nodos = pd.read_csv('../INPUTS/nodos_componente_1.csv', sep=',', index_col="Id")
arcos = pd.read_csv('../INPUTS/arcos_componente_1.csv', sep=',', index_col="Id")


In [12]:
# Funciones
def construir_red(nodos, arcos, dirigida=True):
    if dirigida == True:
        G=nx.DiGraph()
    else:
        G=nx.Graph()
    G.add_nodes_from(nodos.index)
    for nodo in G.nodes:
        for atr in nodos:
            G.nodes[nodo][atr]=nodos[atr][nodo]
    G.add_edges_from([tuple(x) for x in arcos[['SOURCE','TARGET']].to_numpy()])
    return G
G = construir_red(nodos, arcos)

def analizar_top_nodos(analisis_red):
    nodos = []
    for resultado in analisis_red['cálculos para el top 20'].keys():
        for nodo in analisis_red['cálculos para el top 20'][resultado].keys():
            if nodo not in nodos:
                nodos.append(nodo)
            else:
                None

    inventario_nodos = pd.DataFrame(nodos).rename(columns={0:"top_nodos"}).sort_values("top_nodos").reset_index(drop=True)

    for resultado in analisis_red['cálculos para el top 20'].keys():
        inventario_nodos[resultado] = [analisis_red.get("cálculos para el top 20").get(resultado).get(int(ele),np.nan) for ele in inventario_nodos["top_nodos"]]
    return inventario_nodos

def analisis_general_red(graph_networkx, top_nodos=20):
    G = graph_networkx
    G_no_dirigida = G.to_undirected()
    
    analisis_red = {}
    # estadísticas generales de la red
    analisis_red["estadisticas generales"] = {}
    analisis_red["estadisticas generales"]["Es dirigida"] = G.is_directed()
    analisis_red["estadisticas generales"]["Número nodos"] = G.number_of_nodes()
    analisis_red["estadisticas generales"]["Número de aristas"] = G.number_of_edges()
    analisis_red["estadisticas generales"]["Reciprocidad de la red"] = nx.reciprocity(G) # Reciprocidad de la red
    analisis_red["estadisticas generales"]["asortatividad"] = nx.degree_assortativity_coefficient(G) # asortatividad
    analisis_red["estadisticas generales"]["densidad"] = nx.density(G) # Qué proporción de de conexiones entre los nodos existen, esto cambia si es no dirigida

    analisis_red["estadisticas generales"]["fuertemente conectada"] = nx.is_strongly_connected(G) # Puede ser porque sea dirigida o porque hay más de una componente
    analisis_red["estadisticas generales"]["está conectada"] =  G_no_dirigida.is_directed() if G.is_directed() else nx.is_connected(G) # Puede ser porque sea dirigida o porque hay más de una componente
    # Esta medida se ajusta para el análisis de la primera componente
    analisis_red["estadisticas generales"]["diámetro"] = nx.diameter(G) if nx.is_strongly_connected(G) else nx.diameter(G_no_dirigida) # En la primera componente se puede explicar porque hay nodos que solamente reciben (pueden existir otros casos)

    # cálculos sobre la red
    analisis_red["cálculos sobre la red"] = {}
    ## Análisis de las componentes
    
    analisis_red["cálculos sobre la red"]["componentes_de_la_red"] = {i: j for i, j in enumerate(reversed(sorted(list(nx.connected_components(G_no_dirigida)), key=len)))}
    analisis_red["estadisticas generales"]["estadísticas de los componentes"] = {ele:len(analisis_red["cálculos sobre la red"]["componentes_de_la_red"] [ele]) for ele in analisis_red["cálculos sobre la red"]["componentes_de_la_red"].keys()}

    analisis_red["cálculos sobre la red"]["grados"] = G.degree()
    analisis_red["cálculos sobre la red"]["arcos que salen de nodos"] = G.out_degree()
    analisis_red["cálculos sobre la red"]["arcos que llegan a los nodos"] = G.in_degree()
    analisis_red["cálculos sobre la red"]["PageRank"] = nx.pagerank(G)
    analisis_red["cálculos sobre la red"]["centralidad_eigen_vector"] = nx.eigenvector_centrality(G)
    analisis_red["cálculos sobre la red"]["intermediacion"] = nx.betweenness_centrality(G)
    analisis_red["cálculos sobre la red"]["cercania-closeness"] = nx.closeness_centrality(G)
    analisis_red["cálculos sobre la red"]["clustering_dirigido"] = nx.clustering(G) if G.is_directed() else "La red es no dirigida"
    analisis_red["cálculos sobre la red"]["clustering_no_dirigido"] = nx.clustering(G_no_dirigida)


    # resultados para el top
    analisis_red[f"cálculos para el top {top_nodos}"] = {}
    analisis_red[f"cálculos para el top {top_nodos}"]["Nodos más conectados"] = {ele[0]:ele[1] for ele in sorted(dict(G.degree()).items(), key=lambda x: x[1], reverse=True)[:top_nodos]} 
    analisis_red[f"cálculos para el top {top_nodos}"]["nodos con mas arcos que salen"] = {ele[0]:ele[1] for ele in sorted(dict(G.out_degree()).items(), key=lambda x: x[1], reverse=True)[:top_nodos]}
    analisis_red[f"cálculos para el top {top_nodos}"]["nodos con mas arcos que reciben"] = {ele[0]:ele[1] for ele in sorted(dict(G.in_degree()).items(), key=lambda x: x[1], reverse=True)[:top_nodos]}
    analisis_red[f"cálculos para el top {top_nodos}"]["Nodos más PageRank"] = {ele[0]:ele[1] for ele in sorted(nx.pagerank(G).items(), key=lambda x: x[1], reverse=True)[:top_nodos]}
    analisis_red[f"cálculos para el top {top_nodos}"]["nodos_mas_centrales_eigen_vector"] = {ele[0]:ele[1] for ele in sorted(nx.eigenvector_centrality(G).items(), key=lambda x: x[1], reverse=True)[:top_nodos]}
    analisis_red[f"cálculos para el top {top_nodos}"]["nodos_con_mas_intermediacion"] = {ele[0]:ele[1] for ele in sorted(nx.betweenness_centrality(G).items(), key=lambda x: x[1], reverse=True)[:top_nodos]}
    analisis_red[f"cálculos para el top {top_nodos}"]["nodos_con_mas_cercania"] = {ele[0]:ele[1] for ele in sorted(nx.closeness_centrality(G).items(), key=lambda x: x[1], reverse=True)[:top_nodos]}
    analisis_red[f"cálculos para el top {top_nodos}"]["nodos_con_mas_clustering_dirigido"] = {ele[0]:ele[1] for ele in sorted(nx.clustering(G).items(), key=lambda x: x[1], reverse=True)[:top_nodos]} if G.is_directed() else "La red es no dirigida"
    analisis_red[f"cálculos para el top {top_nodos}"]["nodos_con_mas_clustering_no_dirigido"] = {ele[0]:ele[1] for ele in sorted(nx.clustering(G_no_dirigida).items(), key=lambda x: x[1], reverse=True)[:top_nodos]}

    analisis_top = analizar_top_nodos(analisis_red)
    return analisis_red, analisis_top

analisis_red, analisis_top = analisis_general_red(G)

def crear_red_fuertemente_conectada(nxgraph):
    G = nxgraph
    # Encuentra los componentes fuertemente conexos
    sccs = nx.strongly_connected_components(G)
    # Une los componentes fuertemente conexos en un nuevo grafo
    G_scc = nx.compose_all([G.subgraph(c) for c in sccs])
    # Ahora puedes usar G_scc para cálculos que requieren un grafo fuertemente conexo



def exportar_json(diccionario,ruta):
    with open(ruta, "w") as archivo:
        json.dump(diccionario, archivo)

def generar_tabla_componentes (resultado_componentes):
    componentes_red = []
    for componente in resultado_componentes.keys():
        df = pd.DataFrame(resultado_componentes[componente]).rename(columns={0:"nodo"})
        df["componente"] = componente
        componentes_red.append(df)
    
    concatenado = pd.concat(componentes_red).reset_index(drop=True)
    return concatenado



In [14]:
exportar_json(analisis_red["estadisticas generales"],"resultados_primera_componente.json")
analisistop_con_metadato = analisis_top.merge(nodos,how="left", left_on="top_nodos",right_on="Id")
analisistop_con_metadato.to_excel("analisistop_con_metadato_primera_componente.xlsx",index=False)
